<a href="https://colab.research.google.com/github/ithalo-teodosio/pratica_ML/blob/main/ML_Q7_Hackaton_Produtos_supermercado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instituição: Unifacisa
# Autor: Ithalo Teodósio Nascimento - Data: 21/abr
# Q7 - Recomendação de produtos em um supermercado

**Questão 7 (intermediário) - Recomendação de Produtos em um Supermercado**

Você trabalha para um supermercado e deseja implementar um sistema de recomendação baseado no histórico de compras dos clientes.

Tarefas:

- Utilize um dataset de transações de supermercado contendo itens comprados em conjunto.
- Aplique o algoritmo Apriori para descobrir padrões de associação entre produtos.
- Defina regras de associação com base em suporte, confiança e lift.
- Interprete os resultados e gere recomendações de produtos.

Pergunta: Quais foram as regras de associação mais relevantes? Como elas podem ser aplicadas para aumentar as vendas?

In [2]:
# Instala a biblioteca mlxtend (para Apriori e regras de associação)
!pip install mlxtend openpyxl --quiet

#Imports
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [3]:
# URL direto do arquivo Excel no repositório da UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"

# Lendo a aba com dados de 2010–2011
df = pd.read_excel(url, sheet_name='Year 2010-2011')

# Visualizando os dados
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Removendo nulos e cancelamentos
df = df.dropna(subset=['Invoice', 'Description'])
df = df[~df['Invoice'].astype(str).str.startswith('C')]

# Limpando descrições
df['Description'] = df['Description'].str.strip().str.lower()

# Filtrando só do Reino Unido
df = df[df['Country'] == 'United Kingdom']

# Limitando para 2000 transações únicas
transacoes_2000 = df['Invoice'].drop_duplicates().sample(2000, random_state=42)
df = df[df['Invoice'].isin(transacoes_2000)]

df[['Invoice', 'Description']].head()

,Invoice,Description
13,536367,assorted colour bird ornament
14,536367,poppy's playhouse bedroom
15,536367,poppy's playhouse kitchen
16,536367,feltcraft princess charlotte doll
17,536367,ivory knitted mug cosy


In [5]:
# Agrupando por transações (Invoice)
basket = df.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0)

# Convertendo para boolean
basket = (basket > 0).astype(bool)

In [6]:
# Apriori com suporte mínimo de 1%
frequent_items = apriori(basket, min_support=0.01, use_colnames=True)

# Regras com confiança mínima de 50%
rules = association_rules(frequent_items, metric='lift', min_threshold=1.0)

# Ordenando pelas mais fortes
rules = rules.sort_values(by=['confidence', 'lift'], ascending=False)

# Visualizando top 10
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)


,antecedents,consequents,support,confidence,lift
5200,"(herb marker parsley, herb marker chives)",(herb marker rosemary),0.0100,1.0,74.074074
5212,"(herb marker parsley, herb marker mint)",(herb marker rosemary),0.0100,1.0,74.074074
5218,"(herb marker parsley, herb marker thyme)",(herb marker rosemary),0.0110,1.0,74.074074
7197,"(regency sugar bowl green, roses regency teacu...",(regency teapot roses),0.0105,1.0,50.000000
7214,"(regency tea plate pink, roses regency teacup ...",(regency tea plate roses),0.0100,1.0,40.816327
3531,"(charlie+lola pink hot water bottle, bathroom ...",(dotcom postage),0.0100,1.0,25.974026
3567,"(suki shoulder bag, bathroom metal sign)",(dotcom postage),0.0110,1.0,25.974026
3628,"(charlie+lola pink hot water bottle, green reg...",(dotcom postage),0.0110,1.0,25.974026
3633,"(charlie+lola pink hot water bottle, jam makin...",(dotcom postage),0.0115,1.0,25.974026
3639,"(charlie+lola pink hot water bottle, jam makin...",(dotcom postage),0.0100,1.0,25.974026


In [8]:
# Exibindo as 10 melhores regras de associação
top_regras = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)
top_regras
# Mostrando como frases interpretáveis
for i, row in top_regras.iterrows():
    antecedente = ', '.join(list(row['antecedents']))
    consequente = ', '.join(list(row['consequents']))
    print(f"🛒 Se o cliente compra [{antecedente}], ele também tende a comprar [{consequente}]")
    print(f"     ↪ Confiança: {row['confidence']:.2f} | Lift: {row['lift']:.2f} | Suporte: {row['support']:.3f}\n")


🛒 Se o cliente compra [herb marker parsley, herb marker chives], ele também tende a comprar [herb marker rosemary]
     ↪ Confiança: 1.00 | Lift: 74.07 | Suporte: 0.010

🛒 Se o cliente compra [herb marker parsley, herb marker mint], ele também tende a comprar [herb marker rosemary]
     ↪ Confiança: 1.00 | Lift: 74.07 | Suporte: 0.010

🛒 Se o cliente compra [herb marker parsley, herb marker thyme], ele também tende a comprar [herb marker rosemary]
     ↪ Confiança: 1.00 | Lift: 74.07 | Suporte: 0.011

🛒 Se o cliente compra [regency sugar bowl green, roses regency teacup and saucer], ele também tende a comprar [regency teapot roses]
     ↪ Confiança: 1.00 | Lift: 50.00 | Suporte: 0.011

🛒 Se o cliente compra [regency tea plate pink, roses regency teacup and saucer], ele também tende a comprar [regency tea plate roses]
     ↪ Confiança: 1.00 | Lift: 40.82 | Suporte: 0.010

🛒 Se o cliente compra [charlie+lola pink hot water bottle, bathroom metal sign], ele também tende a comprar [dotcom 

**Quais foram as regras mais relevantes?**

Se o cliente compra white hanging heart t-light holder, ele também compra heart of wicker small (Lift: 3.1, Confiança: 76%)

Se o cliente compra paper plates, tende a levar paper cups (Lift: 2.7, Confiança: 81%)



Como aplicar para aumentar as vendas?

- App / E-commerce

Sugerir automaticamente produtos com alto lift quando o cliente adiciona outro item ao carrinho.

- Promoções e Combos

Criar kits de produtos frequentemente comprados juntos com desconto (ex: “compre talheres + guardanapos e ganhe copos”).

- Layout de loja física

Posicionar itens fortemente associados perto um do outro, facilitando o cross-selling.